In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import re
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import matplotlib.pyplot as plt
import string
import collections


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA


In [60]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wvanbuskir002\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wvanbuskir002\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wvanbuskir002\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [61]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html"
page = requests.get(URL, verify = False)
soup = BeautifulSoup(page.content, "html.parser")

In [62]:
URL = 'https://2hy8ik84z1.execute-api.us-east-1.amazonaws.com/default/machine-learning'
page = requests.get(URL, verify = False)
page.content

b'"Hello from Lambda!"'

In [63]:
table_names = soup.find(title="Table showing list of executed inmates")
df = pd.read_html(str(table_names))[0]

In [64]:
df.shape

(575, 10)

In [65]:
df['last_first'] = df['Last Name'].apply(lambda row : row.lower()) + df['First Name'].apply(lambda row: row.lower())

In [66]:
name_list = df['last_first']

Pull each final statement:

In [67]:
statement_list = []
for name in name_list:
    url_offender = f'https://www.tdcj.texas.gov/death_row/dr_info/{name}last.html'
    page_offender = requests.get(url_offender, verify = False)
    soup_offender = BeautifulSoup(page_offender.content, "html.parser")
    try:
        title = soup_offender.body.find(text=re.compile('Last Statement:'))
        statement = title.find_next('p')
        statement_list.append(statement.text)
    except:
        statement_list.append('ERROR')

In [68]:
statement_list[0]

'I would like to thank my  Jesus Christ my Lord and Savior. I would like to thank all these people in my  life and that aided me in this journey. I would also like to thank the Texas  Department of Criminal Justice inmate Field Minister program True Foster and  Garcia for aiding me in my journey. To Ms. Walker’s family I pray my death will  bring you peace.'

In [69]:
for index, row in df.iterrows():
    name = row['last_first']
    url_offender = f'https://www.tdcj.texas.gov/death_row/dr_info/{name}last.html'
    page_offender = requests.get(url_offender, verify = False)
    soup_offender = BeautifulSoup(page_offender.content, "html.parser")
    try:
        title = soup_offender.body.find(text=re.compile('Last Statement:'))
        statement = title.find_next('p')
        df.at[index, 'statement'] = statement.text
    except:
        df.at[index, 'statement'] = 'ERROR'

In [70]:
df.head()

,Execution,Link,Link.1,Last Name,First Name,TDCJNumber,Age,Date,Race,County,last_first,statement
0,575,Inmate Information,Last Statement,Chanthakoummane,Kosol,999529,41,8/17/2022,Other,Collin,chanthakoummanekosol,I would like to thank my Jesus Christ my Lord...
1,574,Inmate Information,Last Statement,Buntion,Carl,993,78,4/21/2022,White,Harris,buntioncarl,Ok to start with I would like to thank everyon...
2,573,Inmate Information,Last Statement,Rhoades,Rick,999049,57,9/28/2021,White,Harris,rhoadesrick,No statement was made.
3,572,Inmate Information,Last Statement,Hummel,John,999567,45,6/30/2021,White,Tarrant,hummeljohn,"Yes, when they lay me down to sleep, for I am..."
4,571,Inmate Information,Last Statement,Jones,Quintin,999379,41,5/19/2021,Black,Tarrant,jonesquintin,I would like to thank all of the supporting p...


In [71]:
df['length'] = df['statement'].map(lambda x: len(x))
df.head()

,Execution,Link,Link.1,Last Name,First Name,TDCJNumber,Age,Date,Race,County,last_first,statement,length
0,575,Inmate Information,Last Statement,Chanthakoummane,Kosol,999529,41,8/17/2022,Other,Collin,chanthakoummanekosol,I would like to thank my Jesus Christ my Lord...,358
1,574,Inmate Information,Last Statement,Buntion,Carl,993,78,4/21/2022,White,Harris,buntioncarl,Ok to start with I would like to thank everyon...,496
2,573,Inmate Information,Last Statement,Rhoades,Rick,999049,57,9/28/2021,White,Harris,rhoadesrick,No statement was made.,23
3,572,Inmate Information,Last Statement,Hummel,John,999567,45,6/30/2021,White,Tarrant,hummeljohn,"Yes, when they lay me down to sleep, for I am...",277
4,571,Inmate Information,Last Statement,Jones,Quintin,999379,41,5/19/2021,Black,Tarrant,jonesquintin,I would like to thank all of the supporting p...,584


In [72]:
## Statement Cleaning
df['statement'] = df['statement'].str.replace(r'\\r','')

c:\Users\wvanbuskir002\.conda\envs\kedro-PY37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [73]:
statements_to_remove = ['ERROR', 'This inmate declined to make a last statement.', 'None']

df = df[~df.statement.isin(statements_to_remove)]

In [74]:
df.to_csv('statements.csv')

PermissionError: [Errno 13] Permission denied: 'statements.csv'

In [ ]:
df.shape

(415, 13)

## Clustering

In [ ]:
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    text = text.translate(str.maketrans('','',string.punctuation))
    text = text.translate(str.maketrans('','','1234567890'))
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens
 

def cluster_texts(texts, clusters=3):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.5,
                                 min_df=0.1,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters)
    km_model.fit(tfidf_model)
 
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering

In [ ]:
clusters_7 = cluster_texts(statement_list, 7)

c:\Users\wvanbuskir002\.conda\envs\kedro-PY37\lib\site-packages\sklearn\feature_extraction\text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'arent', 'becaus', 'befor', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'dure', 'ha', 'hadnt', 'hasnt', 'havent', 'hi', 'isnt', 'mightnt', 'mustnt', 'neednt', 'onc', 'onli', 'ourselv', 'shant', 'shouldnt', 'shouldv', 'thatll', 'themselv', 'thi', 'veri', 'wa', 'wasnt', 'werent', 'whi', 'wont', 'wouldnt', 'youd', 'youll', 'yourselv', 'youv'] not in stop_words.
  % sorted(inconsistent)


In [ ]:
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df)
    distortions.append(kmeanModel.inertia_)
# Plotting the distortions of K-Means
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

[10,
 16,
 22,
 25,
 41,
 76,
 80,
 83,
 85,
 98,
 108,
 114,
 120,
 126,
 144,
 145,
 153,
 156,
 166,
 170,
 173,
 180,
 181,
 189,
 192,
 193,
 197,
 202,
 205,
 213,
 215,
 216,
 217,
 218,
 225,
 226,
 228,
 242,
 249,
 255,
 260,
 263,
 267,
 271,
 272,
 277,
 278,
 283,
 286,
 293,
 298,
 302,
 303,
 309,
 311,
 314,
 322,
 331,
 332,
 346,
 351,
 356,
 364,
 365,
 366,
 369,
 370,
 371,
 374,
 375,
 381,
 382,
 383,
 385,
 388,
 389,
 390,
 391,
 393,
 396,
 397,
 398,
 399,
 401,
 403,
 404,
 417,
 418,
 419,
 423,
 425,
 431,
 442,
 448,
 455,
 458,
 459,
 460,
 462,
 466,
 467,
 468,
 469,
 470,
 471,
 473,
 474,
 477,
 479,
 481,
 482,
 486,
 490,
 493,
 498,
 499,
 500,
 505,
 506,
 508,
 510,
 511,
 513,
 514,
 515,
 516,
 517,
 518,
 519,
 520,
 521,
 522,
 523,
 524,
 525,
 527,
 530,
 531,
 532,
 538,
 541,
 543,
 545,
 546,
 547,
 548,
 549,
 550,
 552,
 553,
 554,
 556,
 562,
 567,
 571,
 572,
 573]

In [ ]:
statement_list[clusters[2]]

TypeError: list indices must be integers or slices, not list

In [ ]:
cluster_2 = [ statement_list[i] for i in clusters[2]]

In [ ]:
cluster_2;

NameError: name 'cluster_2' is not defined